In [ ]:
import pickle
from pathlib import Path
import pandas as pd 
import os
import syss

In [ ]:
import data_set_generate_helper_functions as ghf

In [ ]:
sys.executable

# Path to import the data 

In [ ]:
path_to_import = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/0.data_Sets/1.lateral_medial_6"

# Path where to save it 

In [ ]:
path_to_save= "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis_1"

# Update depending on analysis: 

In [ ]:
seq_str = '0a1b2b3j4abcd5a6a7a'
HowManyBack=1
drop2blocks = "yes"
drop2lastblocks = "no"

# Folder to save the data: 

In [ ]:
folderName="right_dual_color_photo_full_data_after_QC" #folder where the plot data_set will be saved 
#folderName="left_dual_color_photo_full_data_set_after_inspection" #folder where the plot data_set will be saved 

In [ ]:
full_path_to_save = path_to_save+"/"+folderName+"/"+seq_str
full_path_to_save

# import the data table: 

In [ ]:
#name of xlsx file that has the sessions we want to run: 
MasterSheetName= '3.lateral_medial_6_lateral_channels_final'

In [ ]:
Mouse_Date_FileName = pd.read_excel(path_to_save+'/'+folderName+"/"+MasterSheetName+'.xlsx') #mdf is mouse data file 

In [ ]:
Mouse_Date_FileName

In [ ]:
#Mouse_Date_FileName = Mouse_Date_FileName.iloc[0:45,:]

In [ ]:
#Mouse_Date_FileName

# Import the full data structure for each session: 

In [ ]:
def extract_data_object_data_lick(Mouse_Date_FileName,path):
    data_lick_data_set = []
    counter = 1 
    
    l_mouse = list(Mouse_Date_FileName["mouse"]) 
    l_date_ = list(Mouse_Date_FileName.apply(lambda x: generate_date_(x["date"]),axis=1))  
    l_date_and_name = list(Mouse_Date_FileName.apply(lambda x: generate_name_and_date(x["date"],x["mouse"]),axis=1))
    
    numer_of_sessions_dataset = len(l_mouse)
    
    for (mouse, date_, date_and_name) in zip(l_mouse, l_date_, l_date_and_name):
        print ("session number"+" "+str(counter)+" "+"was imported (out of"+" "+str(numer_of_sessions_dataset)+")")
        data_lick = pd.read_excel(path+"/"+mouse+"/"+date_and_name+"/"+mouse+"_dataLick_label.xlsx")
        data_lick_data_set.append(data_lick)  
        counter +=1
        
    return (data_lick_data_set)

In [ ]:
def generate_date_(Date):
    return('20'+str(Date)[:2]+'_'+str(Date)[2:4]+'_'+str(Date)[4:6])

def generate_name_and_date(Date,Mouse):
    return ('20'+str(Date)[:2]+'_'+str(Date)[2:4]+'_'+str(Date)[4:6]+'__'+Mouse)

In [ ]:
%%time
behavior_data_lick_data_set = extract_data_object_data_lick(Mouse_Date_FileName,path_to_import)

extract_data_object_data_lick(Mouse_Date_FileName,path)

# A function to help loop on each sessioon with various functions 

In [ ]:
def session_looper(data_set,function):
    organized_data_lick_list=[]
    for session in data_set:
        result = function(session)
        organized_data_lick_list.append(result)
    return organized_data_lick_list

In [ ]:
def clean_behavior_data_lick (behavior_data_lick):        
    if behavior_data_lick.columns.tolist() == ['iBlock', 'iTrial', 'iSpout', 'sTrial_start', 'sTrial_end', 'tState']:
        pass
    else:
        #Shifting the names on the columns, so each column has the right label. (raw data has these labels shifted)
        behavior_data_lick=behavior_data_lick.rename(columns={"nTrial": "iBlock", "iBlock":"iTrial", 
                                                              "iTrial":"iSpout","iSpout":"sTrial_start", 
                                                              "sTrial_start":"sTrial_end", "sTrial_end":"tState_new"})
        #I will delete the extra column
        behavior_data_lick=behavior_data_lick.drop("tState", 1)
        behavior_data_lick=behavior_data_lick.rename(columns={"tState_new":"tState"})
        
    #this is to delete the first and last rows which have no lick data  
    behavior_data_lick=behavior_data_lick[1: -1]
    #Drop everything before first trial
    behavior_data_lick= behavior_data_lick[behavior_data_lick.iBlock>0]
    return behavior_data_lick

In [ ]:
#add nTrial to dataLick
def get_nTrial_DataLick_list(session):
    
    block=1
    nTrial= 0
    l=[]
    
    for row in session.iterrows(): 
        
        (iblock, iTrial)= (row[1]["iBlock"], row[1]["iTrial"])
        
        if iblock!=block:
            block+=1
            nTrial+=(iTrial)
            l.append((nTrial))
            
        else:
            if iTrial>nTrial:
                nTrial+=(iTrial-nTrial)
                l.append((nTrial))
            elif iTrial<nTrial:
                nTrial+= iTrial - (session.at[row[0]-1, "iTrial"])
                l.append((nTrial))
            else:
                l.append((nTrial))
                
    session["nTrial"]= l
    return session

In [ ]:
behavior_data_lick_data_set_clean = session_looper (behavior_data_lick_data_set,clean_behavior_data_lick)

In [ ]:
%time
behavior_data_lick_data_set_clean = session_looper (behavior_data_lick_data_set_clean,get_nTrial_DataLick_list)

In [ ]:
def drop_first_two_blocks(behavior_data_lick):
    if drop2blocks=="yes":
        behavior_data_lick = behavior_data_lick[behavior_data_lick['iBlock'] > 2]
    
    if drop2lastblocks=="yes":
        lastblock = behavior_data_lick['iBlock'].values.tolist()[-1]
        SecondToLast = lastblock-1
        behavior_data_lick = behavior_data_lick[behavior_data_lick.iBlock<SecondToLast]

    return behavior_data_lick

In [ ]:
behavior_data_lick_data_set_clean = session_looper (behavior_data_lick_data_set_clean,drop_first_two_blocks)

In [ ]:
#behavior_data_lick_data_set_clean[0]

In [ ]:
data_set = behavior_data_lick_data_set_clean.copy()

In [ ]:
#data_set[0].to_csv("sample_data_lick_pickle.csv")

In [ ]:
%%time
root = Path(full_path_to_save)
d = "data_object_data_lick"+"_"+seq_str+'.pickle'
my_path = root / d 
print (my_path)
my_file = open(my_path, 'wb')
my_file = pickle.dump((data_set),my_file)